# Práctico 11: Alineación de imágenes usando SIFT

* Capturar dos imágenes con diferentes vistas del mismo objeto.
* Computar puntos de interés y descriptores en ambas imágenes.
* Establecer matches entre ambos conjuntos de descriptores.
* Eliminar matches usando criterio de Lowe.
* Computar una homografía entre un conjunto de puntos y el otro.
* Aplicar la homografía sobre una de las imágenes y guardarla en otra (mezclarla con un alpha de 50 %).

In [1]:
import numpy as np
import cv2
import matplotlib.pyplot as plt

In [5]:
MIN_MATCH_COUNT = 10

img1 = cv2.imread("img12.jpg")   # Leemos la imagen 1
img2 = cv2.imread("img11.jpg")   # Leemos la imagen 2

cp_img1 = img1.copy()
cp_img2 = img2.copy()

dscr = cv2.xfeatures2d.SIFT_create()

kp1, des1 = dscr.detectAndCompute(img1, None)
kp2, des2 = dscr.detectAndCompute(img2, None)

cv2.drawKeypoints(cp_img1, kp1, cp_img1)
cv2.drawKeypoints(cp_img2, kp2, cp_img2,) 

cv2.namedWindow("Img1")
cv2.imshow("Img1", cp_img1)
cv2.namedWindow("Img2")
cv2.imshow("Img2", cp_img2)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [6]:
matcher = cv2.BFMatcher(cv2.NORM_L2)
matches = matcher.knnMatch(des1, des2, k=2)

# Apply ratio test
good = []
for m,n in matches:
    if m.distance < 0.7*n.distance:
        good.append(m)
        
if(len(good) > MIN_MATCH_COUNT):
    src_pts = np.float32([kp1[m.queryIdx].pt for m in good]).reshape(-1, 1, 2)
    dst_pts = np.float32([kp2[m.trainIdx].pt for m in good]).reshape(-1, 1, 2)
    
    H, mask = cv2.findHomography(dst_pts, src_pts, cv2.RANSAC, 5.0)

img3 = cv2.drawMatches(img1, kp1, img2, kp2, good, None, flags=cv2.DRAW_MATCHES_FLAGS_NOT_DRAW_SINGLE_POINTS) 
img3 = cv2.resize(img3, (int(img3.shape[1]/2), int(img3.shape[0]/2)))
cv2.imshow("Matches", img3)
cv2.waitKey(0)
cv2.destroyAllWindows()

wimg2 = cv2.warpPerspective(img2, H, img2.shape[:2][::-1])

alpha = 0.5
blend = np.array(wimg2 * alpha + img1 * (1 - alpha), dtype=np.uint8)
        
cv2.imshow("Resultado", blend)
cv2.waitKey(0)
cv2.destroyAllWindows()